In [21]:
# 기본
import pandas as pd
import numpy as np

# 최단거리 계산
from haversine import haversine
import geopandas as gpd

### 화재지역 전처리

#### main_code에서 확인했을 때, 지역명이 같은 지역은 매송면만 존재합니다.
#### 중복 지역을 피해주며, 수질등급이 똑같기 때문에 하나만 사용해주었습니다.

In [29]:
aa = pd.read_excel('/Users/home/Desktop/대국민 물 공모전/소방청_연간화재.xlsx')
a1 = pd.read_excel('/Users/home/Desktop/대국민 물 공모전/클러스터2.xlsx')

In [30]:
a1

,Unnamed: 0,지역명,lon,lat,시설명,수질등급
0,5,경기도 성남시 분당구,127.078305,37.350319,낙생,5
1,55,경기도 용인시 처인구,127.306111,37.191672,용담,5
2,89,경기도 화성시 매송면,126.913700,37.253208,어천,5
3,48,경기도 여주시 흥천면,127.559260,37.309390,신근,5
4,60,경기도 이천시 설성면,127.534601,37.142115,성호(설성),6
5,12,경기도 시흥시 물왕동,126.831512,37.384862,흥부,5
6,17,경기도 안성시 대덕면,127.240532,37.024469,거청,6
7,71,경기도 평택시 월곡동,127.130301,37.020553,월곡,6
8,92,경기도 화성시 장안면,126.829077,37.129597,버들,6
9,24,경기도 안성시 양성면,127.252791,37.105754,노곡,6


In [31]:
aa = aa.rename(columns={'a1':'주소'})
aa = aa.drop_duplicates()

a1 = a1[['지역명','시설명','lon','lat']]

### 최단거리 계산

In [32]:
re_location1 = gpd.GeoDataFrame(aa, geometry=gpd.points_from_xy(aa.lon, aa.lat), crs='epsg:4326')
dr_location1 = gpd.GeoDataFrame(a1, geometry=gpd.points_from_xy(a1.lon, a1.lat), crs='epsg:4326')

def location(lo):
    distances =  dr_location1.to_crs(epsg=5186).geometry.distance(lo)
    loc = dr_location1.iloc[distances.idxmin()]['지역명']
    return loc
re_location1['지역명'] = re_location1.to_crs(epsg=5186).apply(lambda x: location(x.geometry), axis=1)

In [35]:
dis = pd.merge(re_location1,dr_location1,on='지역명',how='left')

In [36]:
from haversine import haversine
li = []

dis['lon_x'] = dis['lon_x'].astype('float')
dis['lat_x'] = dis['lat_x'].astype('float')
dis['lon_y'] = dis['lon_y'].astype('float')
dis['lat_y'] = dis['lat_y'].astype('float')

for i in range(dis.shape[0]):
    hbl = (dis['lon_x'][i],dis['lat_x'][i])
    hcl = (dis['lon_y'][i],dis['lat_y'][i])
    result = haversine(hbl,hcl,unit='km')
    li.append(result)

In [37]:
dis['최단거리'] = li

In [38]:
df1 = dis[['주소','지역명','시설명','최단거리']]
df1 = df1.sort_values('최단거리',ascending=False)

#점수 부여시 저수지 명칭을 가져오기 위해 따로 만들어줍니다.
df1 = df1.rename(columns={'지역명':'저수지위치'})
df1
ddf1 = df1[['주소','시설명','저수지위치','최단거리']]
ddf1

,주소,시설명,저수지위치,최단거리
96,경기도 포천시 영북면,성동,경기도 김포시 월곶면,85.736146
443,경기도 포천시 관인면,성동,경기도 김포시 월곶면,84.533245
94,경기도 포천시 영중면,성동,경기도 김포시 월곶면,80.880894
26,경기도 포천시 창수면,성동,경기도 김포시 월곶면,74.814479
405,경기도 연천군 신서면,성동,경기도 김포시 월곶면,70.245374
...,...,...,...,...
38,경기도 시흥시 물왕동,흥부,경기도 시흥시 물왕동,0.855104
611,경기도 평택시 청룡동,월곡,경기도 평택시 월곡동,0.796776
239,경기도 화성시 매송면,어천,경기도 화성시 매송면,0.517862
670,경기도 성남시분당구 동원동,낙생,경기도 성남시 분당구,0.507004
